In [13]:
%gui wx
#%matplotlib inline

In [14]:
from mayavi import mlab
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os
import sys

In [15]:
sys.path.insert(0, os.path.abspath('../..'))
from UKMovementSensing import hsmm

In [16]:
execfile('config.py')

## Load data

In [17]:
# Import data with states
filenames = os.listdir(states_path)
filenames = [fn for fn in filenames if os.path.isfile(os.path.join(states_path,fn))]
datasets = [pd.read_csv(os.path.join(states_path,fn), encoding='utf-8') for fn in filenames]
print(len(datasets))

103


In [18]:
all_data = pd.concat(datasets)

In [19]:
#Concatenate all data together for analysis
all_data = pd.concat(datasets)

In [20]:
#Define consistent colormap
# One more than the number of states, so we have a color for 'All'
colormap, cmap = hsmm.get_color_map(all_data.state.nunique()+1)

In [29]:
#Define consistent colormap
# One more than the number of states, so we have a color for 'All'
colormap, cmap = hsmm.get_color_map(all_data.state.nunique()+1)
lutcmap = np.ones((256, 4))
nrlevels = int(np.round(255.0/nr_states))
for k in range(nr_states):
    lutcmap[k*nrlevels:k*nrlevels+nrlevels] = colormap[k]
lutcmap *= 255
lutcmap = np.array(lutcmap, dtype='uint8')

## Investigate distribution of angles

In [21]:
plt.boxplot([all_data['anglex'], all_data['angley'], all_data['anglez']]);
plt.show()

## Visualize states

In [22]:
#exampledata = datasets[3]
exampledata = all_data 
exampledata.shape

(1511567, 14)

In [23]:
anglex = np.array(exampledata['anglex'])
angley = np.array(exampledata['angley'])
anglez = np.array(exampledata['anglez'])
acc = np.array(exampledata['acceleration']) + 1
state = np.array(exampledata['state'])

In [24]:
nr_states = state.max()+1

In [25]:
radtodegree = 180/np.pi
theta = anglex/radtodegree
psi = angley/radtodegree
phi = anglez/radtodegree

In [26]:
x = np.sin(theta)
y = np.sin(psi)
z = np.tan(phi)*np.sqrt(x**2 + y**2)
#radius = np.sqrt(a_x**2 + a_y**2 + a_z**2)
#x = a_x/radius
#y = a_y/radius
#z = a_z/radius

In [27]:
a_x = x*acc
a_y = y*acc
a_z = z*acc
#a_x = np.array(exampledata.roll_med_acc_x)
#a_y = np.array(exampledata.roll_med_acc_y)
#a_z = np.array(exampledata.roll_med_acc_z)

In [28]:
nmax = 100000
mlab.points3d(x[:nmax], y[:nmax], z[:nmax], mode='point')

In [30]:
fig = mlab.points3d(x[:nmax], y[:nmax], z[:nmax], state[:nmax], mode='point')
fig.module_manager.scalar_lut_manager.lut.table = lutcmap
cb = mlab.colorbar(nb_labels=nr_states, label_fmt='%.0f')
mlab.draw()

In [31]:
sphere = mlab.points3d(0, 0, 0, scale_mode='none',
                                scale_factor=2,
                                color=(0.67, 0.77, 0.93),
                                resolution=50,
                                opacity=0.2)
fig = mlab.points3d(a_x[:nmax], a_y[:nmax], a_z[:nmax], state[:nmax], vmax=nr_states, mode='point')
fig.module_manager.scalar_lut_manager.lut.table = lutcmap
cb = mlab.colorbar(nb_labels = nr_states, label_fmt='%.0f') #